# 1. Data preprocessing

* 用 `numpy.isnan` 檢查有沒有非法的數字，False 就表示沒有


In [ ]:
print( numpy.isnan((pd_data_train).any()) )
print( numpy.isnan((pd_data_test).any()) )

""" output
Date           False
Open Price     False
Close Price    False
High Price     False
Low Price      False
Volume         False
dtype: bool
Date           False
Open Price     False
Close Price    False
High Price     False
Low Price      False
Volume         False
dtype: bool
"""

* 因為預測只會用到 `Data`、`Close Price` 所以只針對這兩列做處理

`Data`：將格式改成 **yyyymmdd**，因此實做出 switch 功能，將月份轉成數字，再重組字串

`Close Price`：需要的只是與前一天的比較結果，而每份資料的第一筆是 **1月1日** 無法得知，所以要去除，
    隔週就以上一週最後一天比較（就是跟前一筆比）
    最後再將結果存入新的 dataframe 中
    
對 train, test set 做相同動作

In [ ]:
# Use lambda to implement switch
switcher_month = {
    'Jan': '01',
    'Feb': '02',
    'Mar': '03',
    'Apr': '04',
    'May': '05',
    'Jun': '06',
    'Jul': '07',
    'Aug': '08',
    'Sep': '09',
    'Oct': '10',
    'Nov': '11',
    'Dec': '12',
}
pd_data_train_y = numpy.zeros(pd_data_train.shape[0], dtype = 'int')

for i in range(0, pd_data_train.shape[0]) :
    tmp = pd_data_train['Date'][i]
    pd_data_train.at[i, 'Date'] = tmp[7:11] + switcher_month.get(tmp[3:6]) + tmp[0:2]
    if i != 0:
        prev = pd_data_train.at[i-1, 'Close Price']
        cur = pd_data_train.at[i, 'Close Price']
        if prev > cur :
            pd_data_train_y[i] = 0
        else :
            pd_data_train_y[i] = 1

pd_data_train_x = pd_data_train[['Date']]
pd_data_train_x = pd_data_train_x.drop([0])

pd_data_train_y = pd.DataFrame(pd_data_train_y, columns=['result'])
pd_data_train_y = pd_data_train_y.drop([0])

# 2. highest classification accuracy in this dataset

三種方法做出來的準確度都一樣，且猜測的部份都是猜 1，若看機率，發現 1 的機率大概0.52多

主要原因大概就是長久來看股價是一直往上升的，這點把資料視覺化就一目了然，

只看這種二元的結果基本上不管用什麼做，結果都會一樣。

若換成其他 dataset 同樣是股票且都有往上爬的趨勢結果應該不會改變，

若是某一隻股價漲跌是斷斷續續的，那結果一定不一樣

# 3. How to improve classifiers ?


這邊只討論 **是否會上升的機率** 與 **預測股價**，二元預測沒有什麼討論空間，

可以改用時間序列相關的 classifier ，其中使用宏觀角度去預測股價，

可能會比用回歸法預測還準確，

因為股價是會受到現實影響（e.g. XXX說參選總統，XX股價就無預警下跌），

只用先前數據去預測用想的就覺得很不實際，不管怎麼調整參數，都有一定上限，

或者是用機器學習去分析新聞資訊，再配合網路爬蟲，

耗費大量功夫或許可做到與現實事件結合的股價預測程式。